# Training model and deploy it into WANDB

In [1]:
import os
from pathlib import Path

p = Path.cwd()
os.chdir(p.parent)
os.getcwd()

'/root/repos/lightning/HousePricing'

In [2]:
from dotenv import dotenv_values
import os

envs = ["secret.env", "fit.env"]

for fenv in envs:
    file = os.path.join("env", fenv)
    config = dotenv_values(file)  # load sensitive variables
    print(config.keys())
    for c, v in config.items():
        os.environ[c] = v

odict_keys(['WANDB_API_KEY'])
odict_keys(['WANDB_NAME', 'WANDB_NOTEBOOK_NAME', 'BATCH_SIZE', 'MAX_EPOCHS', 'PATIENCE', 'LEARNING_RATE', 'VALIDATION_SIZE'])


In [3]:
import wandb
import os

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import torch

torch.cuda.is_available()

True

In [5]:
import torch

torch.cuda.device_count()

1

In [6]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [7]:
import os

max(1, os.cpu_count() - 1)

11

In [8]:
import os
import wandb
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from src.model import NeuralNetwork
from src.data import HousePricingDataModule
from src.utils.utility import fake_name

project_name = os.environ["WANDB_NAME"]

patience = int(os.environ["PATIENCE"])
max_epochs = int(os.environ["MAX_EPOCHS"])
batch_size = int(os.environ["BATCH_SIZE"])
learning_rate = float(os.environ["LEARNING_RATE"])
validation_size = float(os.environ["VALIDATION_SIZE"])
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

# Preparing data to be used
data_module = HousePricingDataModule(
    batch_size=batch_size,
    validation_size=validation_size,
)
data_module.prepare_data()

in_features = data_module.data_features()

# Setting up the training configuration
config = {
    "accelerator": accelerator,
    "max_epochs": max_epochs,
    "patience": patience,
    "lr": learning_rate,
    "batch_size": batch_size,
    "in_features": in_features,
    "validation_size": validation_size,
}

artifacts_dir = dict()

for i in range(3):

    run_name = fake_name()

    wandb.init(
        job_type="training",
        name=run_name,
        project=project_name,
        config=config,
    )

    print(f"[INFO]: Run configuration: {config}")

    # Defining the model to be training
    model = NeuralNetwork(input_size=wandb.config["in_features"], lr=wandb.config["lr"])

    checkpoint_name = f"{wandb.run.name}-{wandb.run.id}"

    # Defining the logger instance the lighning will use as default logging
    logger = WandbLogger(checkpoint_name=checkpoint_name)

    # Define how the model registry work
    checkpoint_callback = ModelCheckpoint(
        every_n_epochs=1,
        monitor="val_loss",
        mode="min",
        save_top_k=2,
        filename="house_pricing-{epoch:02d}-{val_loss:.2f}",
    )

    # Defining early stop configuration
    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        mode="min",
        verbose=True,
        patience=wandb.config["patience"],
    )

    # Defines the training instance
    trainer = Trainer(
        accelerator=wandb.config["accelerator"],
        max_epochs=wandb.config["max_epochs"],
        logger=logger,
        log_every_n_steps=1,
        callbacks=[checkpoint_callback, early_stopping_callback],
    )

    trainer.fit(model, datamodule=data_module)

    artifacts_dir[run_name] = checkpoint_callback.best_model_path

    wandb.finish()

[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Run configuration: {'accelerator': 'gpu', 'max_epochs': 100, 'patience': 10, 'lr': 0.001, 'batch_size': 256, 'in_features': 244, 'validation_size': 0.1}
[INFO]: Input size: 244


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...
[INFO]: Setting up fit dataset/s
[INFO]: Train dataset size: 1166
[INFO]: Validation dataset size: 129


...

Metric val_loss improved by 4514240.000 >= min_delta = 0.0. New best score: 883367104.000


Epoch 99: 100%|██████████| 5/5 [00:00<00:00, 35.17it/s, v_num=cqbp]

Metric val_loss improved by 2912064.000 >= min_delta = 0.0. New best score: 880455040.000
`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 5/5 [00:00<00:00, 31.30it/s, v_num=cqbp]


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,▇█▇█▇▆▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_loss,█████▇▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
train_loss,1502593664.0
trainer/global_step,499
val_loss,880455040.0


In [9]:
artifacts_dir

{'Kayla_SN0K08HK': './lightning_logs/ydlqfd0c/checkpoints/house-pricing-epoch=99-val_loss=885112896.00.ckpt',
 'Brian_6OFMOICS': './lightning_logs/imp8z3cs/checkpoints/house-pricing-epoch=99-val_loss=864983424.00.ckpt',
 'Timothy_7MCBOSZC': './lightning_logs/wibicqbp/checkpoints/house-pricing-epoch=99-val_loss=880455040.00.ckpt'}

In [14]:
import pandas as pd

artifacts_df = pd.DataFrame(columns=["RunExperimentName", "BestModelPath"])
artifacts_df["RunExperimentName"] = artifacts_dir.keys()
artifacts_df["BestModelPath"] = artifacts_dir.values()

artifacts_df

,RunExperimentName,BestModelPath
0,Kayla_SN0K08HK,./lightning_logs/ydlqfd0c/checkpoints/house-pr...
1,Brian_6OFMOICS,./lightning_logs/imp8z3cs/checkpoints/house-pr...
2,Timothy_7MCBOSZC,./lightning_logs/wibicqbp/checkpoints/house-pr...


In [15]:
artifacts_df.to_csv("artifacts.csv", index=None)